In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
# Download and unzip the Book-Crossing dataset
!wget -O BX-CSV-Dump.zip http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
!unzip -o BX-CSV-Dump.zip


--2025-06-20 11:24:36--  http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
Resolving www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)... 132.230.105.133
Connecting to www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)|132.230.105.133|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-06-20 11:24:37 ERROR 404: Not Found.

Archive:  BX-CSV-Dump.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of BX-CSV-Dump.zip or
        BX-CSV-Dump.zip.zip, and cannot find BX-CSV-Dump.zip.ZIP, period.


In [5]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip -o book-crossings.zip > /dev/null
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
import pandas as pd

# Load datasets with proper encoding and delimiter
books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
users = pd.read_csv('BX-Users.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


--2025-06-20 11:31:56--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  40.2MB/s    in 0.6s    

2025-06-20 11:31:57 (40.2 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

/tmp/ipython-input-5-917554458.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


In [7]:
# Standardized load with proper column renaming and types

df_books = pd.read_csv(
    'BX-Books.csv',
    sep=';',
    encoding='latin-1',
    on_bad_lines='skip',
    usecols=['ISBN', 'Book-Title', 'Book-Author'],
    dtype=str
)
df_books.columns = ['isbn', 'title', 'author']

df_ratings = pd.read_csv(
    'BX-Book-Ratings.csv',
    sep=';',
    encoding='latin-1',
    on_bad_lines='skip',
    usecols=['User-ID', 'ISBN', 'Book-Rating'],
    dtype={'User-ID': int, 'ISBN': str, 'Book-Rating': float}
)
df_ratings.columns = ['user', 'isbn', 'rating']


In [8]:
print("Total ratings:", df_ratings.shape[0])
print("Unique users:", df_ratings['user'].nunique())
print("Unique books in ratings:", df_ratings['isbn'].nunique())
print("Unique books in books file:", df_books['isbn'].nunique())


Total ratings: 1149780
Unique users: 105283
Unique books in ratings: 340556
Unique books in books file: 271379


In [9]:
# Filter: users with at least 5 ratings
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 5].index)]

# Filter: books with at least 5 ratings
book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 5].index)]

# Print counts
print("Users after filtering:", df_ratings['user'].nunique())
print("Books after filtering:", df_ratings['isbn'].nunique())


Users after filtering: 21915
Books after filtering: 39702


In [10]:
# Use minimal filtering so we don't lose all data
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 5].index)]

book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 5].index)]

# Print the count after filtering
print("Users after filtering:", df_ratings['user'].nunique())
print("Books after filtering:", df_ratings['isbn'].nunique())


Users after filtering: 16009
Books after filtering: 38190


In [11]:
# Merge with books
merged_df = pd.merge(df_ratings, df_books, on='isbn')

# Create pivot table (Book × User matrix)
book_pivot = merged_df.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Print shape
print("Pivot shape:", book_pivot.shape)

# Convert to sparse matrix
from scipy.sparse import csr_matrix
book_matrix = csr_matrix(book_pivot.values)


Pivot shape: (32194, 15998)


In [12]:
from sklearn.neighbors import NearestNeighbors

# Fit the KNN model using cosine similarity
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_matrix)

# Save the list of book titles
book_titles = list(book_pivot.index)

# Define the recommendation function
def get_recommends(book_title):
    if book_title not in book_pivot.index:
        return [book_title, []]

    book_idx = book_titles.index(book_title)
    distances, indices = model.kneighbors(book_pivot.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(distances[0])):  # skip the first (it’s the book itself)
        recommended_books.append([book_titles[indices[0][i]], distances[0][i]])

    return [book_title, recommended_books]


In [21]:
book_titles[:50]





[' Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth',
 ' Good Wives: Image and Reality in the Lives of Women in Northern New England, 1650-1750',
 ' Goosebumps Monster Edition 1: Welcome to Dead House, Stay Out of the Basement, and Say Cheese and Die!',
 ' Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback))',
 ' Q-Space (Star Trek The Next Generation, Book 47)',
 ' Q-Zone (Star Trek The Next Generation, Book 48)',
 ' This Place Has No Atmosphere (Laurel-Leaf Books)',
 '!Yo!',
 '$14 In The Bank (Cathy Collection)',
 "'A Hell of a Place to Lose a Cow': An American Hitchhiking Odyssey",
 "'Salem's Lot",
 "'The Moon by Whale Light: And Other Adventures Among Bats, Penguins, Crocodilians, and Whales",
 "'Tis Herself : A Memoir",
 "'Tis The Season (Christmas Anthology): Under the Mistletoe/A Baby for Christmas/Christmas Angel/Home for Christmas",
 '...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR',
 '...Sacrifice of Ang

In [23]:
get_recommends("101 Bug Jokes")


['101 Bug Jokes',
 [['Hard Fall', np.float64(0.36073656523925013)],
  ['Slowing Down to the Speed of Life: How to Create a More Peaceful, Simpler Life from the Inside Out',
   np.float64(0.38605938648507954)],
  ['Return of the Straight Dope', np.float64(0.38605938648507954)],
  ['Mr. Paradise:  A Novel', np.float64(0.408520011648551)],
  ['The Seven Daughters of Eve: The Science That Reveals Our Genetic Ancestry',
   np.float64(0.4159401458814752)]]]